In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****Getting lemmas and metadata together****

In [3]:
%%time
# open file in read mode
with open('/kaggle/input/utf8tokenizedspeeches/TokenizedSpeeches_utf-8.csv', 'r') as read_obj:
    lemmasList = []
    
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader: 
        lemmasList.append(row)
    print(len(lemmasList), 'Rows read')

In [ ]:
%%time
#create dataframe from the lemmas extracted from csv
dictOfLemmas = {'Lemmas': lemmasList}
lemmasDf = pd.DataFrame(dictOfLemmas)

In [ ]:
lemmasDf

In [ ]:
with open('/kaggle/input/hansard-speeches-lemmatized/hansard-speeches-post2010.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
#since index was missing values and didn't match with the lemmasDf index
df = df.reset_index(drop=True)

In [ ]:
df = df.join(lemmasDf)

# Dividing and training corpus before and after the Brexit referendum

In [ ]:
# Split data based on the Brexit referendum event before and after period
eventDate = '2016-06-23 23:59:59'
df_t1 = df[df['date']<= eventDate]
df_t2 = df[df['date']> eventDate]

In [ ]:
print(os.path)

In [ ]:
'''%%time
model1 = gensim.models.Word2Vec.load('../input/models12/model1')
model2 = gensim.models.Word2Vec.load('../input/models12/model2')
'''

In [ ]:
%%time
functools.reduce(smart_procrustes_align_gensim, listOfModels)


In [ ]:
'''%%time
model1 = gensim.models.Word2Vec(df_t1['Lemmas'], min_count=1, vector_size=300, window = 5, sg = 1)
model2 = gensim.models.Word2Vec(df_t2['Lemmas'], min_count=1, vector_size=300, window = 5, sg = 1)


# Saving models 

os.makedirs('./original-models')
model1.save('./original-models/model1')
model2.save('./original-models/model2')
!zip -r file.zip /kaggle/working/original-models'''

# Choosing intersecting vocabulary and Aligning models 

In [ ]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """
    print('Vocab function')
    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)
    print('vocabs',list(common_vocab)[0], list(vocab_m1)[0], list(vocab_m2)[0])
    print('1. Common vocab length', len(common_vocab))

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))
    print('2. Common vocab length', len(common_vocab))
    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
    print('Vocab function returning models with shapes', m1.wv.vectors.shape, m2.wv.vectors.shape)    
    return (m1,m2)

In [ ]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    print('shapes', base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """
    print('Hillow hillow.. In Smart procrustes align gensim function, received',base_embed, other_embed)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    print('a, the shapes received are', in_base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    print(str(len(list(in_base_embed.wv.index_to_key))))
    print(str(len(list(in_other_embed.wv.index_to_key))))

    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
    
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    
    print('b')

    other_vecs = in_other_embed.wv.get_normed_vectors()
    print('c')
    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    print('d')

    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    print('e')

    # another matrix operation
    ortho = u.dot(v) 
    print('f')

    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    print('Procrustes function returning')
    return other_embed

In [ ]:
%%time
# Applying the functions to our models

'''smart_procrustes_align_gensim(model1, model2, words=None)
'''

In [ ]:
%%time

model_df_t1_103 = gensim.models.Word2Vec.load('../input/multiplemodelstrial/model_df_t1_103')
model_df_t1_112 = gensim.models.Word2Vec.load('../input/multiplemodelstrial/model_df_t1_112')
model_df_t1_114 = gensim.models.Word2Vec.load('../input/multiplemodelstrial/model_df_t1_114')
model_df_t1_116 = gensim.models.Word2Vec.load('../input/multiplemodelstrial/model_df_t1_116')
model_df_t1_151 = gensim.models.Word2Vec.load('../input/multiplemodelstrial/model_df_t1_151')



In [ ]:
print('Original lengths',len(model_df_t1_103.wv.index_to_key), len(model_df_t1_112.wv.index_to_key), len(model_df_t1_114.wv.index_to_key),len(model_df_t1_116.wv.index_to_key), len(model_df_t1_151.wv.index_to_key))

In [ ]:
print('lengths',len(model_df_t1_103.wv.index_to_key), len(model_df_t1_112.wv.index_to_key), len(model_df_t1_114.wv.index_to_key),len(model_df_t1_116.wv.index_to_key), len(model_df_t1_151.wv.index_to_key))

In [ ]:
%%time

#listOfModels = [model_df_t1_103,model_df_t1_112,model_df_t1_114,model_df_t1_116, model_df_t1_151]
otherEmbedReturned = smart_procrustes_align_gensim(otherEmbedReturned,model_df_t1_116)

In [ ]:
otherEmbedReturned == model_df_t1_103

In [ ]:
otherEmbedReturned == model_df_t1_112

In [ ]:
model1.wv.similarity("brexit", "leave")


In [ ]:
# after alignment 
model1.wv.similarity("brexit", "leave")


In [ ]:
model2.wv.similarity("brexit", "leave")


In [ ]:
model1.wv.similarity("brexit", "remain")


In [ ]:
model2.wv.similarity("brexit", "remain")


In [ ]:
model1.wv.similarity("remain", "europe")


In [ ]:
model2.wv.similarity("remain", "europe")


# Measuring change in words

In [ ]:
def cosine_similarity(word):
  sc = 1-spatial.distance.cosine(model1.wv[word], model2.wv[word])
  return sc

In [ ]:
%%time
cosine_similarity_df = pd.DataFrame(([w, cosine_similarity(w), model1.wv.get_vecattr(w, "count") , model2.wv.get_vecattr(w, "count") ] for w in model1.wv.index_to_key), columns = ('Word', 'Cosine_similarity', "Frequency_t1", "Frequency_t2"))
# ??
cosine_similarity_df['FrequencyRatio'] = cosine_similarity_df['Frequency_t1']/cosine_similarity_df['Frequency_t2']

In [ ]:
hist = cosine_similarity_df['Cosine_similarity'].hist()


In [ ]:
cosine_similarity_df_sorted = cosine_similarity_df.sort_values(by='Cosine_similarity', ascending=True)
cosine_similarity_df_sorted.shape

In [ ]:
# We should now filter and only consider words that have a decent frequency of usage 
cosine_similarity_df_sorted = cosine_similarity_df_sorted.loc[cosine_similarity_df_sorted['Frequency_t1'] + cosine_similarity_df_sorted['Frequency_t2'] > 50].sort_values(by='Cosine_similarity', ascending = True)


In [ ]:
cosine_similarity_df_sorted.tail(100).Word.tolist()

In [ ]:
len(cosine_similarity_df_sorted.loc[cosine_similarity_df_sorted['Frequency_t1'] + cosine_similarity_df_sorted['Frequency_t2'] > 50].Word.tolist())


In [ ]:
# Plot graph between cosine similarity and frequency ratio to check if there's a relation

In [ ]:
words_chosen = ['brexit', 'leave', 'remain', 'europe', 'eurozone', 'euro', 'UKIP', 'population', 'group', 'community']

In [ ]:
cosine_similarity_chosen = cosine_similarity_df_sorted[cosine_similarity_df_sorted['Word'].isin(words_chosen)]
cosine_similarity_chosen

In [ ]:
#Chosen most working reverse - manually analysed ones with small change, to discuss with Chico

words_chosen_expected_no_change = [ 'population', 'group', 'community', 'said', 'tell', 'told', 'get', 'however', 'year', 'given', 'might']

In [ ]:
cosine_similarity_df_sorted[cosine_similarity_df_sorted['Word'].isin(words_chosen_expected_no_change)]


# Splitting speeches by MPs and time

In [ ]:
# Code to create word2Vec models and save them
# Commenting since we have already done that and are going to load them 
'''%%time
# Lets only consider the MPs who have speeches in both T1 & T2 time periods, intersection between two DFs 

t1List = pd.unique(df_t1['mnis_id']).tolist()
t2List = pd.unique(df_t2['mnis_id']).tolist()

intersectedList = list(set(t1List).intersection(t2List))

print(len(intersectedList))
# 651 MPs to be considered


#  --------- new cell --------

# Now modify original df_t1 and df_t2 DFs to only contain speeches from the common MPs 

df_t1 = df_t1[df_t1['mnis_id'].isin(intersectedList)]
df_t2 = df_t2[df_t2['mnis_id'].isin(intersectedList)]


# ---------new cell ------------

%%time
# New code for dividing corpus by time and MPs 

dictSpeechesByMp = {}

for mpId in intersectedList:
    for dfTime in ['df_t1','df_t2']:
        dfName = dfTime + '_'+ mpId
        if(dfTime == 'df_t1'):
            dictSpeechesByMp[dfName]=df_t1[df_t1['mnis_id']==mpId]
        elif (dfTime == 'df_t2'):
            dictSpeechesByMp[dfName]=df_t2[df_t2['mnis_id']==mpId]

# -----------new cell ----------------
# Do not run, for verification only

len(dictSpeechesByMp.keys()) 
# Result 1302 

df_t1['mnis_id'].value_counts()
df_t2['mnis_id'].value_counts()
# Result is 651 each - hence all covered in our dictionary DFs 

# --------------new cell ---------------

os.makedirs('./models-by-mp-and-time')
 
# --------------new cell ---------------

%%time 
models_folder = './models-by-mp-and-time'

count =1
for dframe in dictSpeechesByMp: 
    #print(dframe)
    if (count<652):
        model = gensim.models.Word2Vec(dictSpeechesByMp[dframe]['Lemmas'], min_count=1, vector_size=300, window = 5, sg = 1)
        modelName ='model_'+ dframe
        print('Saving model number', count, modelName)

        model.save(os.path.join(models_folder, modelName))
    elif(count>651):
        print('651 files done and exiting')
        break
    count = count+1
    #if(count==2):
     #   break;
        
# ------------ new cell --------
%%time
!zip -r file.zip /kaggle/working/models-by-mp-and-time

last -adding: kaggle/working/models-by-mp-and-time/model_df_t2_450 (deflated 8%)'''